In [1]:
import pandas as pd
import math
import gym
import numpy as np
from gym import spaces
from sklearn import preprocessing

/home/archoline/.local/lib/python3.10/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [132]:
df = pd.read_csv('./Binance_BTCUSDT_minute.csv', 
              usecols = 
              ['unix', 
               'date', 
               'symbol', 
               'open', 
               'high', 
               'low', 
               'close', 
               'Volume BTC', 'Volume USDT', 'tradecount'])

In [120]:
Unix Timestamp - This is the unix timestamp or also known as "Epoch Time". Use this to convert to your local timezone
Date - This timestamp is UTC Timezone
Symbol - The symbol for which the timeseries data refers
Open - This is the opening price of the time period
High - This is the highest price of the time period
Low - This is the lowest price of the time period
Close - This is the closing price of the time period
Volume (Crypto) - This is the volume in the transacted Ccy. Ie. For BTC/USDT, this is in BTC amount
Volume Base Ccy - This is the volume in the base/converted ccy. Ie. For BTC/USDT, this is in USDT amount
Trade Count - This is the unique number of trades for the given time period

SyntaxError: invalid syntax (350783561.py, line 1)

In [121]:
def normalize(df):
    result = df.copy()
    for feature_name in ['open', 
               'high', 
               'low', 
               'close', 
               'Volume BTC', 'Volume USDT']:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [122]:
df = normalize(df) #normalizing data
df

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1642730760000,2022-01-21 02:06:00,BTC/USDT,0.554149,0.553656,0.556514,0.554118,8.201588e-06,2.887443e-05,27
1,1642730700000,2022-01-21 02:05:00,BTC/USDT,0.554935,0.554474,0.556542,0.554146,4.045122e-03,1.425156e-02,1333
2,1642730640000,2022-01-21 02:04:00,BTC/USDT,0.555594,0.555256,0.557216,0.554935,4.457450e-03,1.572702e-02,1911
3,1642730580000,2022-01-21 02:03:00,BTC/USDT,0.554178,0.555103,0.556468,0.555594,5.698815e-03,2.009408e-02,2073
4,1642730520000,2022-01-21 02:02:00,BTC/USDT,0.553643,0.554605,0.555946,0.554178,6.103231e-03,2.149369e-02,2654
...,...,...,...,...,...,...,...,...,...,...
1240676,1567965660000,2019-09-08 18:01:00,BTC/USDT,0.096381,0.095383,0.097878,0.096381,0.000000e+00,0.000000e+00,0
1240677,1567965600000,2019-09-08 18:00:00,BTC/USDT,0.096381,0.095383,0.097878,0.096381,0.000000e+00,0.000000e+00,0
1240678,1567965540000,2019-09-08 17:59:00,BTC/USDT,0.096381,0.095383,0.097878,0.096381,7.039989e-08,6.213658e-08,1
1240679,1567965480000,2019-09-08 17:58:00,BTC/USDT,0.096381,0.095383,0.097878,0.096381,0.000000e+00,0.000000e+00,0


In [123]:
import datetime

def as_day(date_str):
    date, hour = date_str.split(' ')
    y,m,d = date.split('-')
    return int(datetime.datetime(int(y), int(m), int(d)).weekday())
def as_hour(date_str):
    date, hour = date_str.split(' ')
    h,_,_ = hour.split(':')
    return int(h)
    

#as_day('2022-01-01 18:21:00')
df['day_of_the_week'] = df['date'].apply(as_day)
df['hour_of_the_day'] = df['date'].apply(as_hour)


In [124]:
df_clean = df.copy()
df_clean.drop(columns=['tradecount', 'symbol', 'unix', 'date'], inplace=True) #Droping unusable columns
df_clean

,open,high,low,close,Volume BTC,Volume USDT,day_of_the_week,hour_of_the_day
0,0.554149,0.553656,0.556514,0.554118,8.201588e-06,2.887443e-05,4,2
1,0.554935,0.554474,0.556542,0.554146,4.045122e-03,1.425156e-02,4,2
2,0.555594,0.555256,0.557216,0.554935,4.457450e-03,1.572702e-02,4,2
3,0.554178,0.555103,0.556468,0.555594,5.698815e-03,2.009408e-02,4,2
4,0.553643,0.554605,0.555946,0.554178,6.103231e-03,2.149369e-02,4,2
...,...,...,...,...,...,...,...,...
1240676,0.096381,0.095383,0.097878,0.096381,0.000000e+00,0.000000e+00,6,18
1240677,0.096381,0.095383,0.097878,0.096381,0.000000e+00,0.000000e+00,6,18
1240678,0.096381,0.095383,0.097878,0.096381,7.039989e-08,6.213658e-08,6,17
1240679,0.096381,0.095383,0.097878,0.096381,0.000000e+00,0.000000e+00,6,17


In [126]:
class BitcoinTradingEnv(gym.Env):
    """A Bitcoin trading environment for OpenAI gym"""
    metadata = {'render.modes': ['live', 'file', 'none']}
    scaler = preprocessing.MinMaxScaler()
    viewer = None 
    def __init__(self, df, lookback_window_size=50, 
                         commission=0.00075,  
                         initial_balance=10000,
                         serial=False):
        super(BitcoinTradingEnv, self).__init__()  
        self.df = df.dropna().reset_index()
        self.lookback_window_size = lookback_window_size
        self.initial_balance = initial_balance
        self.commission = commission
        self.serial = serial  # Actions of the format Buy 1/10, Sell 3/10, Hold, etc.
        self.action_space = spaces.MultiDiscrete([3, 10])  # Observes the OHCLV values, net worth, and trade history
        self.observation_space = spaces.Box(low=0, high=1, shape=(10, 
                        lookback_window_size + 1), dtype=np.float16)
    
    def reset(self):
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.btc_held = 0
        self._reset_session()  
        self.account_history = np.repeat([
            [self.net_worth],
            [0],
            [0],
            [0],
            [0]], self.lookback_window_size + 1, axis=1)  
        self.trades = []
        return self._next_observation()

    def step(self, action):
        current_price = self._get_current_price() + 0.01
        self._take_action(action, current_price)
        self.steps_left -= 1
        self.current_step += 1
        
        if self.steps_left == 0:
            self.balance += self.btc_held * current_price
            self.btc_held = 0
            self._reset_session() 
        
        obs = self._next_observation()
        reward = self.net_worth
        done = self.net_worth <= 0  
        
        return obs, reward, done, {}
    
    def _take_action(self, action, current_price):
        action_type = action[0]
        amount = action[1] / 10  
        btc_bought = 0
        btc_sold = 0
        cost = 0
        sales = 0  
        if action_type < 1:
            btc_bought = self.balance / current_price * amount
            cost = btc_bought * current_price * (1 + self.commission)
            self.btc_held += btc_bought
            self.balance -= cost  
        elif action_type < 2:
            btc_sold = self.btc_held * amount
            sales = btc_sold * current_price  * (1 - self.commission)
            self.btc_held -= btc_sold
            self.balance += sales
        if btc_sold > 0 or btc_bought > 0:
            self.trades.append({
                'step': self.frame_start+self.current_step,
                'amount': btc_sold if btc_sold > 0 else btc_bought,
                'total': sales if btc_sold > 0 else cost,
                'type': "sell" if btc_sold > 0 else "buy"
            })  
        self.net_worth = self.balance + self.btc_held * current_price
        self.account_history = np.append(self.account_history, [
            [self.net_worth],
            [btc_bought],
            [cost],
            [btc_sold],
            [sales]
        ], axis=1)